In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.nn.parallel import DataParallel
import torch
from sklearn import metrics
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import os, json, time
from transformers import RobertaTokenizer, RobertaModel

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
!rm -r DC_data/text/.ipynb_checkpoints/
!rm -r DC_data/labels/.ipynb_checkpoints/

rm: cannot remove 'DC_data/text/.ipynb_checkpoints/': No such file or directory
rm: cannot remove 'DC_data/labels/.ipynb_checkpoints/': No such file or directory


### Data Process

In [4]:
#len should be 1852
len(os.listdir('DC_data/text/')), len(os.listdir("DC_data/labels/"))

(1852, 1852)

In [5]:
for dirpath, dirnames, filenames in os.walk('./DC_data/text/'):
    idx2text = {}
    for filename in filenames:
        if filename.startswith('.'):
            continue
        idx = filename.split('.')[0]
        with open(dirpath + filename) as f:
            abstract = ''
            for sentence in f:
                abstract += sentence
            abstract = abstract.replace('\n', ' ')
            idx2text[idx] = abstract

for dirpath, dirnames, filenames in os.walk('./DC_data/labels/'):
    idx2label = [] 
    for filename in filenames:
        if filename.startswith('.'):
            continue
        idx = filename.split('.')[0]
        with open(dirpath + filename) as f:
            labels = f.readline()
            for label in labels.split('<'):
                if not label or label.isspace():
                    continue
                key_label = label.split('--')[0].strip()
                if key_label == 'NULL':
                    continue
                idx2label.append((idx, key_label))

In [6]:
idx2label = list(set(idx2label))
df = pd.DataFrame(idx2label, columns=['idx', 'label'])
df['text'] = df['idx'].map(idx2text)

In [7]:
label_vals = list(set(df['label'].tolist()))

In [8]:
df = df.groupby(['idx', 'text']).agg({'label': lambda x: list(x)}).reset_index()

In [9]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('label')),
                          columns=mlb.classes_,
                          index=df.index))

In [10]:
df.head()

,idx,text,Activating invasion and metastasis,Avoiding immune destruction,Cellular energetics,Enabling replicative immortality,Evading growth suppressors,Genomic instability and mutation,Inducing angiogenesis,Resisting cell death,Sustaining proliferative signaling,Tumor promoting inflammation
0,11724768,Ghrelin was identified in the stomach as an en...,0,0,0,0,0,0,0,0,1,0
1,11773160,PURPOSE The epidermal growth factor receptor (...,0,0,0,0,1,0,0,0,1,0
2,11774243,Adoptive transfer of immunity against hepatiti...,0,1,0,0,0,0,0,0,0,0
3,11781072,The secretion of immunosuppressive factors lik...,0,1,0,0,0,0,0,1,1,0
4,11791181,To characterize the impact of increased produc...,1,0,0,0,0,0,0,0,1,0


In [10]:
del idx2label, idx2text

### Model Paths

In [11]:
checkpoint_folder4GB = '/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/4GB-checkpoints/'
checkpoint_folder12GB = '/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/12GB-checkpoints'

In [11]:
model_paths = ['bert-base-uncased',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/4GB-checkpoints/model-trained-0-3531-4GB',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/4GB-checkpoints/model-trained-18-67089-4GB/',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/4GB-checkpoints/model-trained-36-130647-4GB/',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/12GB-checkpoints/model-trained-0-10596-12GB/',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/12GB-checkpoints/model-trained-3-42384-12GB/',
'/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/12GB-checkpoints/model-trained-5-63576-12GB/'
              ]

### Create Dataset/Model/Training/Eval loops

In [16]:
class HoCDataset:
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.sentences)
        
    def __getitem__(self, item):
        sentence = str(self.sentences[item])
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )
        ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'label': torch.tensor(self.labels[item], dtype=torch.float)
        } 

    
class BERTClass(nn.Module):
    def __init__(self, model_path):
        super(BERTClass, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(model_path)
        self.out = nn.Linear(768, 10)

    
    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        
        return self.out(output)
    
    
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)


def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d['ids']
        mask = d['mask']
        token_type_ids = d['token_type_ids']
        labels = d['label']
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        labels = labels.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if bi % 50 == 0:
            print(f'bi={bi}, loss={loss}')


def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_labels = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        with torch.no_grad():
            ids = d['ids'].to(device, dtype=torch.long)
            mask = d['mask'].to(device, dtype=torch.long)
            token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
            labels = d['label'].to(device, dtype=torch.long)
          
            outputs = model(ids, mask, token_type_ids)
          
            fin_labels.append(labels.cpu().detach().numpy())
            fin_outputs.append(torch.sigmoid(outputs).cpu().detach().numpy())

    return np.vstack(fin_outputs), np.vstack(fin_labels)

In [22]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
EPOCHS = 4
LEARNING_RATE = 3e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


start = time.perf_counter()

for model_path in model_paths:
    tokenizer_path = ('bert-base-uncased' if model_path == 'bert-base-uncased'\
                  else '../../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt')
    model_name = model_path if model_path == 'bert-base-uncased' else model_path.split('/')[-2].split('.')[0]
    scores = []
    model_stats = {'model_name':model_name,
                   'seeds':[],
                   'batch_size':TRAIN_BATCH_SIZE,
                   'epochs':EPOCHS,
                   'metric':'f1-score',
                   'scores': [],
                   'mean_score':0
                    }
    for num, seed in enumerate([42,43,44,45,46], 1):
        SEED = seed
        model_stats['seeds'].append(SEED)
        label_cols = list(df.drop(['idx', 'text'], axis=1).columns)
        labels = df[label_cols].values

        df_train, df_test, labels_train, labels_test = train_test_split(
            df, labels, test_size=0.2, random_state=SEED)


        tokenizer = transformers.BertTokenizer.from_pretrained(tokenizer_path)
        model = BERTClass(model_path)
        model = DataParallel(model)
        print(f'Using model {model_name}, with tokenizer {tokenizer_path}')
        model.to(device)

        train_dataset = HoCDataset(
            sentences=df_train.text.values,
            labels=labels_train,
            tokenizer=tokenizer,
            max_len=MAX_LEN
        )
        train_data_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=TRAIN_BATCH_SIZE,
            shuffle=True
        )

        test_dataset = HoCDataset(
            sentences=df_test.text.values,
            labels=labels_test,
            tokenizer=tokenizer,
            max_len=MAX_LEN
        )
        test_data_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=4,
            shuffle=True,
            drop_last=True
        )

        optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        num_training_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=0,
            num_training_steps=num_training_steps
        )
        for epoch in range(EPOCHS):
            train_loop_fn(train_data_loader, model, optimizer, device, scheduler)
            
        output, target = eval_loop_fn(test_data_loader, model, device)
        preds = np.array(output) >= 0.5
        f1_score_micro = metrics.f1_score(target, preds, average='micro')
        print(f"F1 Score (Micro) = {round(f1_score_micro,4)}")
        model_stats['scores'].append(round(f1_score_micro,6))
        torch.cuda.empty_cache()
        time.sleep(3)
        print(f'Training run {num} completed.')
    
    print('Logging model stats....')
    print()
    final_score = np.round(np.mean(model_stats['scores']), 4)
    model_stats['mean_score'] = final_score
    with open('logs/stats.txt', 'a') as f:
        f.write(json.dumps(model_stats))
        f.write('\n')

end = time.perf_counter() - start
print(f'Total Training/Eval time: {round(end, 2)} seconds')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using model bert-base-uncased, with tokenizer bert-base-uncased
bi=0, loss=0.6724764108657837
bi=50, loss=0.353378027677536
bi=100, loss=0.3245600163936615
bi=150, loss=0.34938469529151917
bi=0, loss=0.27509862184524536
bi=50, loss=0.24954788386821747
bi=100, loss=0.2549605071544647
bi=150, loss=0.22175753116607666
bi=0, loss=0.199954554438591
bi=50, loss=0.2637068033218384
bi=100, loss=0.20407043397426605
bi=150, loss=0.15232689678668976
bi=0, loss=0.1554514765739441
bi=50, loss=0.15801814198493958
bi=100, loss=0.18348319828510284
bi=150, loss=0.18558908998966217
F1 Score (Micro) = 0.7865
Training run 1 completed.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using model bert-base-uncased, with tokenizer bert-base-uncased
bi=0, loss=0.742846667766571
bi=50, loss=0.3756067752838135
bi=100, loss=0.3423362970352173
bi=150, loss=0.29819798469543457
bi=0, loss=0.31681180000305176
bi=50, loss=0.2845384180545807
bi=100, loss=0.23170910775661469
bi=150, loss=0.23327182233333588
bi=0, loss=0.26353582739830017
bi=50, loss=0.21036015450954437
bi=100, loss=0.18905289471149445
bi=150, loss=0.1947965919971466
bi=0, loss=0.2277781069278717
bi=50, loss=0.134521946310997
bi=100, loss=0.1886909306049347
bi=150, loss=0.2373591959476471
F1 Score (Micro) = 0.7908
Training run 2 completed.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Using model model-trained-5-63576-12GB, with tokenizer ../../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt
bi=0, loss=0.6771770715713501
bi=50, loss=0.39519867300987244
bi=100, loss=0.2796936631202698
bi=150, loss=0.15073202550411224
bi=0, loss=0.16502727568149567
bi=50, loss=0.1557181030511856
bi=100, loss=0.15564946830272675
bi=150, loss=0.1173369437456131
bi=0, loss=0.15398333966732025
bi=50, loss=0.09786175191402435
bi=100, loss=0.14127571880817413
bi=150, loss=0.07902289927005768
bi=0, loss=0.08364153653383255
bi=50, loss=0.0764140784740448
bi=100, loss=0.1043485775589943
bi=150, loss=0.08634033799171448
F1 Score (Micro) = 0.8291
Training run 3 completed.


/anaconda/envs/py38_pytorch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1658: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/12GB-checkpoints/model-trained-5-63576-12GB/ were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a Bert

Using model model-trained-5-63576-12GB, with tokenizer ../../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt
bi=0, loss=0.7335861325263977
bi=50, loss=0.3691302239894867
bi=100, loss=0.2727179527282715
bi=150, loss=0.17970715463161469
bi=0, loss=0.17992575466632843
bi=50, loss=0.12280433624982834
bi=100, loss=0.1305527687072754
bi=150, loss=0.13935764133930206
bi=0, loss=0.13676123321056366
bi=50, loss=0.1763909012079239
bi=100, loss=0.11906283348798752
bi=150, loss=0.07294964045286179
bi=0, loss=0.09469934552907944
bi=50, loss=0.1062656044960022
bi=100, loss=0.046425338834524155
bi=150, loss=0.09574195742607117
F1 Score (Micro) = 0.8217
Training run 4 completed.


/anaconda/envs/py38_pytorch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1658: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/12GB-checkpoints/model-trained-5-63576-12GB/ were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a Bert

Using model model-trained-5-63576-12GB, with tokenizer ../../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt
bi=0, loss=0.7019864916801453
bi=50, loss=0.4045860469341278
bi=100, loss=0.26260167360305786
bi=150, loss=0.26338788866996765
bi=0, loss=0.21676340699195862
bi=50, loss=0.2085028886795044
bi=100, loss=0.08165328204631805
bi=150, loss=0.11604722589254379
bi=0, loss=0.11582460254430771
bi=50, loss=0.10508110374212265
bi=100, loss=0.10520865768194199
bi=150, loss=0.0810474082827568
bi=0, loss=0.05983774736523628
bi=50, loss=0.09452822059392929
bi=100, loss=0.08544166386127472
bi=150, loss=0.09806782007217407
F1 Score (Micro) = 0.823
Training run 5 completed.
Logging model stats....

Total Training/Eval time: 6506.26 seconds
